## Conexão com o banco de dados e import de bibliotecas

In [1]:
import pandas as pd
import psycopg2

In [2]:
conn = psycopg2.connect(host="localhost",database="Qr-bus", user="postgres", password="bd2018")


## Relatorio 1

### - Objetivo: Relatório que informe ao usuário seu histórico de viagens, com dias e horários em que usou o sistema, assim como as linhas utilizadas.

 codigo para obtenção do resultado:


In [3]:
usuario_id ='3'
relatorio_historico = pd.read_sql_query("""
                select data, hora, numero_linha from passagem 
                inner join usuario 
                on (usuario.id_usuario = """+usuario_id+");",conn)

In [4]:
relatorio_historico

,data,hora,numero_linha
0,2018-10-08,07:30:00,815
1,2018-10-08,12:00:00,815
2,2018-10-09,07:30:00,815
3,2018-10-09,12:00:00,815
4,2018-10-10,07:30:00,815
5,2018-10-10,12:00:00,815
6,2018-10-11,07:30:00,815
7,2018-10-11,12:00:00,815
8,2018-10-12,07:30:00,815
9,2018-10-12,12:00:00,815


## Relatorio 2

### - Objetivo: Relatório que informe as linhas mais utilizadas e as menos utilizadas (por todos os usuários) dependendo do horário.

 codigo para obtenção do resultado:


In [5]:
relatorio_linhas_utilizacao = pd.read_sql_query("""
                select numero_linha,count(id_passagem) as qtd_passagens,hora 
                as numero_de_passagens 
                from passagem 
                GROUP by numero_linha, hora;""",conn)

In [6]:
relatorio_linhas_utilizacao

,numero_linha,qtd_passagens,numero_de_passagens
0,815,5,12:00:00
1,507,5,18:00:00
2,507,5,12:30:00
3,815,5,07:30:00


## Relatorio 3

### - Objetivo: Lista direcionada ao usuário informando as opções de linhas para sair de sua origem

 codigo para obtenção do resultado:


In [7]:
relatorio_linhas = pd.read_sql_query("""
                select usuario.nome,itinerario.numero_linha from ponto 
                inner join usuario on (ponto.id_logradouro = usuario.id_logradouro) 
                inner join itinerario_ponto on (ponto.id_ponto = itinerario_ponto.id_ponto)
                inner join itinerario on (itinerario.id_itinerario= itinerario_ponto.id_itinerario)
                group by (usuario.nome,itinerario.numero_linha)""",conn)

In [8]:
relatorio_linhas

,nome,numero_linha
0,Ingrid Brum,507
1,Ingrid Brum,501


## Relatorio 4

### - Objetivo: Relatório de linhas que passam por um determinado logradouro

 codigo para obtenção do resultado:


In [9]:
relatorio_linhas = pd.read_sql_query("""
                select itinerario.numero_linha from ponto 
                inner join itinerario_ponto on (ponto.id_ponto = itinerario_ponto.id_ponto)
                inner join itinerario on (itinerario_ponto.id_itinerario = itinerario.id_itinerario )
                inner join logradouro on (ponto.id_logradouro = logradouro.id_logradouro)
                where(logradouro.desc_logradouro = 'Nossa Senhora Da Penha')
                group by (itinerario.numero_linha) """,conn)

In [10]:
relatorio_linhas

,numero_linha
0,501
1,507


## Relatorio 5

### - Objetivo: Relatório direcionado ao usuário para controle de seu saldo, com quantidade de viagens feitas por dia/semana/mês, valor de cada viagem e gasto total no período definido.

 codigo para obtenção do resultado:


In [11]:
relatorio_passagem_porAno = pd.read_sql_query("""
                select date_part('year',passagem.data) as data_ano,
                sum(valor) as soma_passagens from passagem
                group by(data_ano)""",conn)

In [12]:
relatorio_passagem_porMes = pd.read_sql_query("""
               select date_part('month',passagem.data) as data_mes,
               sum(valor) as soma_passagens from passagem
               group by(data_mes)""",conn)

In [13]:
relatorio_passagem_porDia = pd.read_sql_query("""
                select date_part('day',passagem.data) as data_dia,
                sum(valor) as soma_passagens from passagem
                group by(data_dia)""",conn)

In [14]:
relatorio_passagem_porAno


,data_ano,soma_passagens
0,2018.0,"R$ 68,00"


In [15]:
relatorio_passagem_porMes

,data_mes,soma_passagens
0,10.0,"R$ 68,00"


In [16]:
relatorio_passagem_porDia

,data_dia,soma_passagens
0,16.0,"R$ 6,80"
1,8.0,"R$ 6,80"
2,18.0,"R$ 6,80"
3,12.0,"R$ 6,80"
4,10.0,"R$ 6,80"
5,11.0,"R$ 6,80"
6,15.0,"R$ 6,80"
7,9.0,"R$ 6,80"
8,17.0,"R$ 6,80"
9,19.0,"R$ 6,80"
